<a href="https://colab.research.google.com/github/Estu-bekti/Estu-bekti/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import cv2
import os

def create_patches(image_path, patch_size=64):
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    patches = []

    # Loop untuk memecah gambar menjadi patch ukuran patch_size x patch_size
    for y in range(0, height, patch_size):
        for x in range(0, width, patch_size):
            patch = image[y:y+patch_size, x:x+patch_size]
            if patch.shape[0] == patch_size and patch.shape[1] == patch_size:
                patches.append(patch)

    return patches


In [20]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd

class TrashPatchDataset(Dataset):
    def __init__(self, root_dir, csv_file, transform=None, patch_size=64):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(csv_file)
        self.transform = transform
        self.patch_size = patch_size

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")
        label = int(self.annotations.iloc[idx, 1])

        # Memecah gambar menjadi patch-patch kecil
        patches = create_patches(img_path, self.patch_size)

        # Apply transformasi ke setiap patch
        if self.transform:
            patches = [self.transform(Image.fromarray(patch)) for patch in patches]

        return patches, label


In [21]:
import torchvision.models as models
import torch.nn as nn

# Load ResNet model yang telah dilatih sebelumnya
model = models.resnet50(pretrained=True)

# Modifikasi layer terakhir untuk menyesuaikan dengan jumlah kelas sampah
num_classes = 5  # misalnya 5 kelas sampah
model.fc = nn.Linear(model.fc.in_features, num_classes)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 118MB/s]


In [22]:
import torch.optim as optim

# Hyperparameter
num_epochs = 10
learning_rate = 0.001
batch_size = 32

# Transformasi untuk augmentasi data
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Dataset dan DataLoader
dataset = TrashPatchDataset(root_dir='path/to/images', csv_file='path/to/labels.csv', transform=transform)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Optimizer dan loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Training loop
model.train()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (patches, labels) in enumerate(data_loader):
        patches = torch.cat(patches, dim=0).to(device)  # Menggabungkan patch dalam satu batch
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(patches)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 9:  # Print setiap 10 batch
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}], Loss: {running_loss / 10:.4f}")
            running_loss = 0.0

print("Training selesai!")


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/labels.csv'